In [11]:
# ===============================================
# LIMPIEZA DE COMENTARIOS + DIVISIÓN EN ORACIONES
# ===============================================

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import spacy

# --- 1. Descarga de recursos necesarios ---
nltk.download('punkt')
nltk.download('stopwords')

# Modelo de español de SpaCy
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    import spacy.cli
    spacy.cli.download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

# --- 2. Cargar tu dataset ---
file_path = "data/reviews_filmaffinity_limpio.csv"  # cambia según tu ruta
df = pd.read_csv(file_path)

# Ajusta nombres de columnas según tu dataset
GENRE_COLUMN = 'film_genre'
REVIEW_TEXT_COLUMN = 'review_text'

# --- 3. Limpieza general del texto ---
def clean_text_general(text):
    """
    Limpieza general del texto para análisis sintáctico.
    """
    if not isinstance(text, str):
        text = str(text)

    # Pasar a minúsculas
    text = text.lower()

    # Arreglar codificación y quitar tildes
    text = (text.replace('Ã¡', 'á').replace('Ã©', 'é').replace('Ã­', 'í')
                    .replace('Ã³', 'ó').replace('Ãº', 'ú').replace('Ã±', 'ñ')
                    .replace('â€™', "'").replace('â€œ', '"').replace('â€', '"'))
    
    # Normalizar sin tildes
    text = (text.replace('á', 'a').replace('é', 'e').replace('í', 'i')
                    .replace('ó', 'o').replace('ú', 'u').replace('ñ', 'n'))

    # Quitar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Quitar caracteres especiales y números
    text = re.sub(r'[^a-z\s]', '', text)

    # Quitar espacios repetidos
    text = re.sub(r'\s+', ' ', text).strip()

    return text


# Aplicar limpieza
df['cleaned_review_general'] = df[REVIEW_TEXT_COLUMN].apply(clean_text_general)

print("\n--- Comentarios limpios (primeras 3 filas) ---")
print(df[['cleaned_review_general']].head(3))


# --- 4. Dividir los comentarios en oraciones ---
def dividir_en_oraciones(text):
    """
    Divide un texto en oraciones cortas usando SpaCy.
    """
    doc = nlp(text)
    oraciones = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 0]
    return oraciones

# Crear una nueva columna con las oraciones
df['sentences'] = df['cleaned_review_general'].apply(dividir_en_oraciones)

# --- 5. Expandir las oraciones en una nueva tabla ---
# Cada oración tendrá su propio registro (útil para el análisis sintáctico posterior)
df_sentences = df.explode('sentences', ignore_index=True)

print("\n--- Ejemplo de comentarios divididos en oraciones ---")
print(df_sentences[['cleaned_review_general', 'sentences']].head(10))


# --- 6. Guardar el resultado limpio y dividido ---
output_path = "comentarios_limpios_y_divididos.csv"
df_sentences.to_csv(output_path, index=False, encoding='utf-8')
print(f"\nArchivo guardado como: {output_path}")



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Angelica\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Angelica\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



--- Comentarios limpios (primeras 3 filas) ---
                              cleaned_review_general
0  la mayor virtud de esta pelicula es su existen...
1  no soy un experto cinefilo pero pocas veces me...
2  si no eres un incondicional del humor estilo t...

--- Ejemplo de comentarios divididos en oraciones ---
                              cleaned_review_general  \
0  la mayor virtud de esta pelicula es su existen...   
1  no soy un experto cinefilo pero pocas veces me...   
2  si no eres un incondicional del humor estilo t...   
3  no se que esta pasando si la gente se deja lle...   
4  pero cuando amanecey me quedo solosiento en el...   
5  la llegada de rafa a euskadi es como ponerse a...   
6  el nivel del cine y lo peor la capacidad intel...   
7  es triste ver una pelicula como esta y escucha...   
8  puedo entender que torrente i y ii y lo imposi...   
9  ahivalohostia txomin que monton de gente riend...   

                                           sentences  
0  la mayor v

In [14]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Descargar recursos de NLTK (solo la primera vez)
nltk.download('stopwords')
nltk.download('punkt')

# --- Cargar tu CSV ---
df = pd.read_csv("data/reviews_filmaffinity_limpio.csv")

# Verifica el nombre de tu columna con los comentarios
columna_texto = "review_text"  # cámbiala si tu dataset usa otro nombre

# --- Función de limpieza ---
def limpiar_texto(texto):
    if pd.isna(texto):
        return ""
    
    # 1️⃣ Pasar a minúsculas
    texto = texto.lower()
    
    # 2️⃣ Eliminar caracteres no deseados (mantener comas, puntos y signos de interrogación/exclamación)
    texto = re.sub(r'[^a-záéíóúüñ0-9,.;:!?¿¡\s]', '', texto)
    
    # 3️⃣ Eliminar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    # 4️⃣ (Opcional) Eliminar stopwords — útil si haces conteos de palabras,
    # pero NO si vas a hacer análisis sintáctico con spaCy
    stop_words = set(stopwords.words('spanish'))
    palabras = [p for p in texto.split() if p not in stop_words]
    texto = " ".join(palabras)
    
    return texto

# --- Aplicar limpieza ---
df["cleaned_review_general"] = df[columna_texto].apply(limpiar_texto)

# --- Mostrar las primeras filas limpias ---
print("\n--- Comentarios limpios (primeras 3 filas) ---")
print(df["cleaned_review_general"].head(3))

# --- Guardar CSV limpio ---
df.to_csv("reviews_limpio.csv", index=False, encoding="utf-8-sig")
print("\n✅ Archivo 'reviews_limpio.csv' generado correctamente.")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Angelica\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Angelica\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!



--- Comentarios limpios (primeras 3 filas) ---
0    mayor virtud película existencia.el hecho poda...
1    experto cinéfilo, pocas veces tan juego sala c...
2    si incondicional humor estilo tele 5.si termin...
Name: cleaned_review_general, dtype: object

✅ Archivo 'reviews_limpio.csv' generado correctamente.


In [23]:
import pandas as pd 

data = pd.read_csv("reviews_limpio.csv")
data.head()

#df['cleaned_review_general']

,film_name,gender,film_avg_rate,review_rate,review_title,review_text,cleaned_review_general
0,Ocho apellidos vascos,Comedia,6.0,3.0,OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO,La mayor virtud de esta película es su existen...,mayor virtud película existencia.el hecho poda...
1,Ocho apellidos vascos,Comedia,6.0,2.0,El perro verde,"No soy un experto cinéfilo, pero pocas veces m...","experto cinéfilo, pocas veces tan juego sala c..."
2,Ocho apellidos vascos,Comedia,6.0,2.0,Si no eres de comer mierda... no te comas esta...,Si no eres un incondicional del humor estilo T...,si incondicional humor estilo tele 5.si termin...
3,Ocho apellidos vascos,Comedia,6.0,2.0,Aida: The movie,"No sé qué está pasando, si la gente se deja ll...","sé pasando, si gente deja llevar modas, si ver..."
4,Ocho apellidos vascos,Comedia,6.0,2.0,UN HOMBRE SOLO (Julio Iglesias 1987),"""Pero cuando amanece,y me quedo solo,siento en...","amanece,y quedo solo,siento fondoun mar vacío,..."


In [25]:
import pandas as pd
import re
import nltk

# Descargar recursos de NLTK (solo la primera vez)
nltk.download('punkt')

# --- Cargar el CSV ---
df = pd.read_csv("data/reviews_filmaffinity_limpio.csv")

columna_texto = "review_text"  # ajusta si tu dataset usa otro nombre

# --- Función de limpieza (sin eliminar stopwords) ---
def limpiar_texto_glc(texto):
    if pd.isna(texto):
        return ""
    
    # 1️⃣ Pasar a minúsculas
    texto = texto.lower()
    
    # 2️⃣ Eliminar caracteres no deseados, PERO mantener puntuación útil
    texto = re.sub(r'[^a-záéíóúüñ0-9,.;:!?¿¡\s]', '', texto)
    
    # 3️⃣ Eliminar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    return texto

# --- Aplicar limpieza ---
df["cleaned_review_general"] = df[columna_texto].apply(limpiar_texto_glc)

# --- Mostrar primeras filas ---
print("\n--- Comentarios limpios (primeras 3 filas) ---")
print(df[["review_text", "cleaned_review_general"]].head(3))

# --- Guardar CSV limpio ---
df.to_csv("reviews_limpio_GLC.csv", index=False, encoding="utf-8-sig")
print("\n✅ Archivo 'reviews_limpio_GLC.csv' generado correctamente.")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Angelica\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!



--- Comentarios limpios (primeras 3 filas) ---
                                         review_text  \
0  La mayor virtud de esta película es su existen...   
1  No soy un experto cinéfilo, pero pocas veces m...   
2  Si no eres un incondicional del humor estilo T...   

                              cleaned_review_general  
0  la mayor virtud de esta película es su existen...  
1  no soy un experto cinéfilo, pero pocas veces m...  
2  si no eres un incondicional del humor estilo t...  

✅ Archivo 'reviews_limpio_GLC.csv' generado correctamente.


In [26]:
import pandas as pd 

data = pd.read_csv("reviews_limpio_GLC.csv")
data.head()

#df['cleaned_review_general']

,film_name,gender,film_avg_rate,review_rate,review_title,review_text,cleaned_review_general
0,Ocho apellidos vascos,Comedia,6.0,3.0,OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO,La mayor virtud de esta película es su existen...,la mayor virtud de esta película es su existen...
1,Ocho apellidos vascos,Comedia,6.0,2.0,El perro verde,"No soy un experto cinéfilo, pero pocas veces m...","no soy un experto cinéfilo, pero pocas veces m..."
2,Ocho apellidos vascos,Comedia,6.0,2.0,Si no eres de comer mierda... no te comas esta...,Si no eres un incondicional del humor estilo T...,si no eres un incondicional del humor estilo t...
3,Ocho apellidos vascos,Comedia,6.0,2.0,Aida: The movie,"No sé qué está pasando, si la gente se deja ll...","no sé qué está pasando, si la gente se deja ll..."
4,Ocho apellidos vascos,Comedia,6.0,2.0,UN HOMBRE SOLO (Julio Iglesias 1987),"""Pero cuando amanece,y me quedo solo,siento en...","pero cuando amanece,y me quedo solo,siento en ..."


In [27]:
import pandas as pd
import spacy
from spacy import displacy

# --- 1️⃣ Cargar modelo de español ---
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    import spacy.cli
    spacy.cli.download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

# --- 2️⃣ Cargar tu CSV limpio ---
df = pd.read_csv("reviews_limpio_GLC.csv")

columna_texto = "cleaned_review_general"

# --- 3️⃣ Dividir en oraciones ---
def dividir_oraciones(texto):
    """Divide un comentario largo en oraciones."""
    if not isinstance(texto, str) or texto.strip() == "":
        return []
    doc = nlp(texto)
    return [sent.text.strip() for sent in doc.sents]

df["oraciones"] = df[columna_texto].apply(dividir_oraciones)

print("\n--- Ejemplo de división en oraciones ---")
print(df[["cleaned_review_general", "oraciones"]].head(2))

# --- 4️⃣ Crear árboles sintácticos ---
def obtener_arbol_sintactico(oracion):
    """Genera una representación textual del árbol sintáctico (GLC) de una oración."""
    doc = nlp(oracion)
    arbol = []
    for token in doc:
        arbol.append(f"{token.text} ({token.dep_}) ← {token.head.text}")
    return "\n".join(arbol)

# Aplicar sobre las primeras 5 oraciones (para no saturar la salida)
ejemplo_oracion = df["oraciones"].explode().dropna().iloc[:5].tolist()
print("\n--- Árboles sintácticos de ejemplo ---\n")
for i, oracion in enumerate(ejemplo_oracion, 1):
    print(f"🟩 Oración {i}: {oracion}")
    print(obtener_arbol_sintactico(oracion))
    print("-"*60)

# --- 5️⃣ (Opcional) Visualización gráfica del árbol ---
# Puedes descomentar esto para visualizar un árbol con displacy
# doc = nlp(ejemplo_oracion[0])
# displacy.serve(doc, style="dep")

# --- 6️⃣ Estadística sintáctica ---
# Contar cuántas oraciones tiene cada comentario
df["num_oraciones"] = df["oraciones"].apply(len)

# Calcular promedios por género (si tu dataset tiene la columna 'gender')
if "gender" in df.columns:
    estadistica = df.groupby("gender")["num_oraciones"].mean().reset_index()
    estadistica.columns = ["Género", "Promedio de oraciones por reseña"]
    print("\n--- Estadística sintáctica (por género) ---")
    print(estadistica)
else:
    print("\nNo se encontró la columna 'gender' para agrupar por género.")

print("\n✅ Análisis sintáctico completado correctamente.")



--- Ejemplo de división en oraciones ---
                              cleaned_review_general  \
0  la mayor virtud de esta película es su existen...   
1  no soy un experto cinéfilo, pero pocas veces m...   

                                           oraciones  
0  [la mayor virtud de esta película es su existe...  
1  [no soy un experto cinéfilo, pero pocas veces ...  

--- Árboles sintácticos de ejemplo ---

🟩 Oración 1: la mayor virtud de esta película es su existencia.el hecho de que podamos jugar con los tópicos más extremos de las identidades patrias la andaluza y la vasca sin que nadie se escandalice ni ponga el grito en el cielo, indica mucho de nuestra madurez como nación pese a quien pese.
la (det) ← virtud
mayor (amod) ← virtud
virtud (nsubj) ← hecho
de (case) ← película
esta (det) ← película
película (nmod) ← virtud
es (cop) ← hecho
su (det) ← existencia.el
existencia.el (cop) ← hecho
hecho (ccomp) ← indica
de (mark) ← jugar
que (mark) ← jugar
podamos (aux) ← jugar
jugar